# mining.py

notebook 版本

## Mining Example
Source: http://www.gurobi.com/resources/examples/mining

一个生产问题，生产哪些产品

This model is an example of a production problem. In production planning problems, choices must be made regarding what resources to use to produce what products. These problems are common across a broad range of manufacturing and mining industries.

使用多周期生产规划5年挖矿计划。公司拥有不同的矿产，生产的数量不同。目标是创建最优化的生产计划，以最大化收益。

In this example we’ll model and solve a multi-period production planning problem. In this case, the application is to optimize mine production across a number of mines over a five year period. The company has different mines, which can be opened, working, or closed. Each mine varies in the quality of its ore and the amount of ore that can be extracted. The quality of ore required to meet production goals varies each year. The aim is to create an optimal production plan for the next 5 years to maximize the total profit, while considering all production capacity, mine restrictions and mine costs.

Note: you can download the model, implemented in Python, here. More information on this type of model can be found in the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams.

H. Paul Williams, Model Building in Mathematical Programming, fifth edition (Page 261-262, 357)

version 1

## Problem Description

TOOD

### Step 1: Import the Gurobi Python Modue and initalize the datastructures.

In [1]:
from gurobipy import *

# tested with
#  Python 2.7.13 :: Anaconda 4.3.1 (x86_64) Gurobi 7.0.2
#  Python 3.5.2 & Gurobi 7.0.1

mines = range(3+1)
years = range(4+1)

Royalties = [5e6, 4e6, 4e6, 5e6]
ExtractLimit = [2e6, 2.5e6, 1.3e6, 3e6]
OreQuality  = [1, .7, 1.5, .5]
BlendedQuality = [0.9, 0.8, 1.2, 0.6, 1.0]
discount = [(1/(1+1/10.0)) ** year for year in years]

mines_limit = 3
sell_price = 10


### Step 2: Create a model and the variables

In [2]:
model = Model('Mining')

out = model.addVars(mines, years, name="output")
quan = model.addVars(years, name="quantity")
work = model.addVars(mines, years, vtype=GRB.BINARY, name="working")
open = model.addVars(mines, years, vtype=GRB.BINARY, name="open")

### Step 3: Add constraints

In [3]:
# At most three mines open
model.addConstrs((work.sum('*',year) <= mines_limit for year in years), "AtMost3Mines")

# Maintain Quality
model.addConstrs(
    (quicksum(OreQuality[mine]*out[mine, year] for mine in mines) == BlendedQuality[year]*quan[year]
     	for year in years), "Quality")

# Quantity produced equals output
model.addConstrs((out.sum('*',year) == quan[year] for year in years), "OutQty")


# Alternatively, this constraint can also be modeled using Gurobi 7.0's new general constraints

for year in years:
    for mine in mines:
        out[mine, year].ub= ExtractLimit[mine]
        model. addGenConstrIndicator(work[mine, year], 0, out[mine, year] == 0, name="ExtractLimit" ) 
        
#Modeled using Gurobi General Constraints
for year in years:
    for mine in mines:
        model. addGenConstrIndicator(work[mine, year], 1, open[mine, year] == 1, name="ExtractLimit" ) 
        
        
years2 = (year for year in years if year < years[-1])
for year in years2:
    for mine in mines:
        model. addGenConstrIndicator(open[mine, year + 1], 1, open[mine, year] == 1, name="SubsequentOpen" ) 

### Step 4: Set objective function

In [4]:
# Maximize Profit
obj = quicksum(sell_price*discount[year]*quan[year] for year in years) \
      - quicksum(Royalties[mine] * discount[year] * open[mine, year]
                 for mine, year in open)
model.setObjective(obj, GRB.MAXIMIZE)

### Step 6: Solve model and Print result

In [5]:
model.optimize()
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

Optimize a model with 15 rows, 65 columns and 70 nonzeros
Model has 56 general constraints
Variable types: 25 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  Objective range  [7e+00, 5e+06]
  Bounds range     [1e+00, 3e+06]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -7.50576e+07
Presolve removed 25 rows and 25 columns
Presolve time: 0.01s
Presolved: 66 rows, 116 columns, 151 nonzeros
Presolved model has 56 SOS constraint(s)
Variable types: 20 continuous, 96 integer (60 binary)

Root relaxation: objective 2.543811e+08, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2.5438e+08    0   16 -7.506e+07 2.5438e+08   439%     -    0s
H    0     0                      -0.0000000 2.5438e+08      -     -    0s
H    0     0                    6.290909e+07 2.5438e+08   304%     -    

## Result
Optimal solution found (tolerance 1.00e-04)

Best objective 1.468619743642e+08, best bound 1.468619743642e+08, gap 0.0000%